In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
# bert = AutoModel.from_pretrained('bert-base-uncased').cuda()

# del model
torch.cuda.empty_cache()

In [3]:
# Load vicuna
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_slug = 'ehartford/Wizard-Vicuna-30B-Uncensored'
# model_slug = "eachadea/vicuna-13b-1.1"


tokenizer = AutoTokenizer.from_pretrained(model_slug)

model = AutoModelForCausalLM.from_pretrained(model_slug, device_map="auto", torch_dtype=torch.float16)

In [2]:
# Vicuna inference
input_text = "Tell me how to become more well read"

vicuna_preamble="A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."

def to_vicuna(text):
#     return vicuna_preamble + f"\nUSER: {text}\nASSISTANT:"
    return  f"USER: {text}\nASSISTANT:"


input_ids = tokenizer(to_vicuna(input_text), return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_length=512, do_sample=True, temperature=0.7)
print(tokenizer.decode(outputs[0]))

<s> USER: Tell me how to become more well read
ASSISTANT: There are a few things you can do to become more well read:

1. Read widely: Try to read books from different genres, authors, and time periods. This will expose you to different styles of writing and help you develop a broader understanding of literature.

2. Set goals: Set a goal for how many books you want to read in a month or a year. This will motivate you to read more and help you track your progress.

3. Join a book club: Joining a book club can be a great way to discuss books with others and discover new authors and genres.

4. Read reviews: Read reviews of books on websites like Goodreads or newspapers to get a sense of what other people thought about a particular book.

5. Keep a diary: Write down your thoughts and feelings about the books you read in a diary or journal. This can help you remember what you read and reflect on how it made you feel.

6. Ask for recommendations: Ask friends, family, or coworkers for book 

In [ ]:
import jsonlines
from tqdm import tqdm

with jsonlines.open('./prompts_augmented.jsonl') as reader:
    prompts = [row for row in reader]
    
continuations = []

In [ ]:
len(prompts)

In [ ]:

for row in tqdm(prompts):
    full_prompt = to_vicuna(row['prompt'])
    input_ids = tokenizer(full_prompt, return_tensors="pt").input_ids.to("cuda")

    outputs = model.generate(input_ids, max_new_tokens=512, do_sample=True, temperature=0.7)
    continuation = tokenizer.decode(outputs[0]).replace(full_prompt, '')
    continuations.append({**row, 'response': continuation})
    
    torch.cuda.empty_cache()
    
with jsonlines.open('./output_augmented_vicuna30uncensored.jsonl','w') as writer:
    writer.write_all(continuations)

In [ ]:
len(continuations)

In [ ]:
continuations[-1]

In [1]:
import torch
torch.cuda.is_available()

True

In [1]:
# Load falcon
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers, torch

# model_slug = 'tiiuae/falcon-7b-instruct'
model_slug = 'tiiuae/falcon-40b-instruct'



tokenizer = AutoTokenizer.from_pretrained(model_slug, padding_side='left')
model = AutoModelForCausalLM.from_pretrained(
    model_slug,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
#     load_in_8bit=True,
    device_map="auto",
)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    batch_size=4,
)



Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusFo

In [2]:
torch.cuda.empty_cache()

In [3]:
import jsonlines
from tqdm import tqdm
from torch.utils.data import Dataset

with jsonlines.open('./prompts_augmented_v2.jsonl') as reader:
    prompts = [row for row in reader]
    
def format_prompt_llama(instruction):
    template = """<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
<</SYS>>

{instruction} [/INST]"""
    return template.format(instruction=instruction)
    
continuations = []
# for row in tqdm(prompts):]
pipeline.tokenizer.pad_token_id = model.config.eos_token_id

class ListDataset(Dataset):
    def __init__(self, original_list):
        self.original_list = original_list

    def __len__(self):
        return len(self.original_list)

    def __getitem__(self, i):
        return self.original_list[i]

ds = ListDataset([row['prompt'] for row in prompts])
torch.cuda.empty_cache()
sequences = []
for seq in tqdm(pipeline(
   ds,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    return_full_text=False,
    # batch_size=4,
), total=len(prompts)):
    sequences.extend(seq)
    torch.cuda.empty_cache()


for x, row in zip(sequences, prompts):
    continuation = x['generated_text']
    continuations.append({**row, 'response': continuation})
    

    
with jsonlines.open('./output_augmented_v2_falcon40.jsonl','w') as writer:
    writer.write_all(continuations)

  0%|                                                                                                                                                                        | 0/2700 [00:00<?, ?it/s]/opt/conda/envs/py310/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
 40%|████████████████████████████████████████████████████████████▍                                                                                          | 1080/2700 [19:23:29<29:05:13, 64.64s/it]


KeyboardInterrupt: 

In [4]:

for x, row in zip(sequences, prompts):
    continuation = x['generated_text']
    continuations.append({**row, 'response': continuation})
    

    
with jsonlines.open('./output_augmented_v2_falcon40_1080.jsonl','w') as writer:
    writer.write_all(continuations)
len(continuations)

1080

In [ ]:
sequences[10]

In [ ]:
import gcsfs

gs = gcsfs.GCSFileSystem(
)

with gs.open('gs://cohere-dev-central-2/tom/granular-eval/batch_controlled/output_augmented_v2_falcon40.jsonl','w') as f:
    writer = jsonlines.Writer(f)
    writer.write_all(continuations)

In [ ]:
prompts[0]

In [ ]:
!gcloud compute instances stop tom-gpu

In [ ]:
del model

In [ ]:
# Load MPT 30
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model_slug = 'mosaicml/mpt-30b-instruct'
# model_slug = "eachadea/vicuna-13b-1.1"




tokenizer = AutoTokenizer.from_pretrained(model_slug, padding_side='left')
model = AutoModelForCausalLM.from_pretrained(
    model_slug,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
#     load_in_8bit=True,
    device_map="auto",
)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    batch_size=4,
)



In [ ]:

import jsonlines
from tqdm import tqdm
from torch.utils.data import Dataset

with jsonlines.open('./prompts_augmented_v2.jsonl') as reader:
    prompts = [row for row in reader]
    
def format_prompt_mpt(instruction):
    template = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n###Instruction\n{instruction}\n\n### Response\n"
    return template.format(instruction=instruction)
    
continuations = []
# for row in tqdm(prompts):]
pipeline.tokenizer.pad_token_id = model.config.eos_token_id

class ListDataset(Dataset):
    def __init__(self, original_list):
        self.original_list = original_list

    def __len__(self):
        return len(self.original_list)

    def __getitem__(self, i):
        return self.original_list[i]

ds = ListDataset([format_prompt_mpt(row['prompt']) for row in prompts])
torch.cuda.empty_cache()
sequences = []
for seq in tqdm(pipeline(
   ds,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    return_full_text=False,
    # batch_size=4,
), total=len(prompts)):
    sequences.extend(seq)
    torch.cuda.empty_cache()


for x, row in zip(sequences, prompts):
    continuation = x['generated_text']
    continuations.append({**row, 'response': continuation})
    

    
with jsonlines.open('./output_augmented_v2_mpt30instruct.jsonl','w') as writer:
    writer.write_all(continuations)

In [ ]:
continuations[0]

In [ ]:
import gcsfs

gs = gcsfs.GCSFileSystem(
)

with gs.open('gs://cohere-dev-central-2/tom/granular-eval/batch_controlled/output_augmented_v2_mpt30instruct.jsonl','w') as f:
    writer = jsonlines.Writer(f)
    writer.write_all(continuations)

In [ ]:
print('Completed!')

!gcloud compute instances stop tom-gpu

In [4]:
# Load MPT 7
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model_slug = 'mosaicml/mpt-7b-instruct'
# model_slug = "eachadea/vicuna-13b-1.1"




tokenizer = AutoTokenizer.from_pretrained(model_slug)
model = AutoModelForCausalLM.from_pretrained(
    model_slug,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
#     load_in_8bit=True,
    device_map="auto",
)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)



The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Instantiating an MPTForCausalLM model from /home/tomhosking_cohere_com/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/1fc4634127ec64a45716003578b9cfae23265849/modeling_mpt.py
You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The model 'MPTForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormFor

In [5]:
import jsonlines
from tqdm import tqdm

with jsonlines.open('./prompts_merged.jsonl') as reader:
    prompts = [row for row in reader]
    
def format_prompt_mpt(instruction):
    template = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n###Instruction\n{instruction}\n\n### Response\n"
    return template.format(instruction=instruction)
    
continuations = []
for row in tqdm(prompts):
    sequences = pipeline(
       format_prompt_mpt(row['prompt']),
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        return_full_text=False
    )
    continuation = sequences[0]['generated_text']
    continuations.append({**row, 'response': continuation})
    
    torch.cuda.empty_cache()
    
with jsonlines.open('./output_merged_mpt7instruct.jsonl','w') as writer:
    writer.write_all(continuations)

 42%|██████████████████████████████████████████████████████████████▏                                                                                    | 317/750 [34:12<46:44,  6.48s/it]


AssertionError: Cannot forward input with seq_len=2049, this model only supports seq_len<=2048

In [ ]:
continuations[0]